In [47]:
import dask.dataframe as dd
import dask.array as da
from dask.distributed import Client, TimeoutError, LocalCluster
from dask.diagnostics import ProgressBar
import matplotlib.pyplot as plt
import pandas as pd
import hvplot.dask
import numpy as np

#code found on stack exchange
#makes a new client if one is not running, runs 6 workers
try:
    client = Client('tcp://localhost:8787', timeout='2s')
except OSError or TimeoutError:
    cluster = LocalCluster(scheduler_port=8787, n_workers=6, ip='localhost')
    client = Client(cluster)
#

client


<Client: 'tcp://127.0.0.1:8787' processes=6 threads=12, memory=16.00 GiB>

In [48]:
#read and print the dataframes

#contains tweets (and time of the tweet) that contain mental health key word
#polled ever 1 minute
tweetdf = dd.read_csv(urlpath='sample_mental_health_tweets.csv', sep='\t' )

#contains counts of tweets for every hour
countdf = dd.read_csv(urlpath='sample_mental_health_tweet_count.csv', sep='\t')

print(tweetdf)

print(countdf)

Dask DataFrame Structure:
                tweet    time
npartitions=1                
               object  object
                  ...     ...
Dask Name: read-csv, 1 tasks
Dask DataFrame Structure:
              timestamp  count
npartitions=1                 
                 object  int64
                    ...    ...
Dask Name: read-csv, 1 tasks


In [49]:

#convert the timestamps to date times
tweetdf['time'] = dd.to_datetime(tweetdf['time'])

countdf['timestamp'] = dd.to_datetime(countdf['timestamp'])


#separate the components of the time for grouping
tweetdf['year'] = tweetdf['time'].dt.year
tweetdf['month'] = tweetdf['time'].dt.month
tweetdf['day'] = tweetdf['time'].dt.day
tweetdf['hour'] = tweetdf['time'].dt.hour
tweetdf['minute'] = tweetdf['time'].dt.minute

countdf['year'] = countdf['timestamp'].dt.year
countdf['month'] = countdf['timestamp'].dt.month
countdf['day'] = countdf['timestamp'].dt.day
countdf['hour'] = countdf['timestamp'].dt.hour

In [50]:
#sort the count values based on ascening tme
countdf = countdf.sort_values(ascending=True, by=['month', 'day', 'hour'])

#make a display string for the x axis of the graph
countdf['ts_display'] = countdf['month'].astype('str') + '/' + countdf['day'].astype('str') + '\t' + countdf['hour'].astype('str') + ':00'

#TODO MAKE THE GRAPH LOOK BETTER
countdf.hvplot(x='ts_display', y='count').opts(xrotation=90, fontscale=0.5)


:Curve   [ts_display]   (count)